### Dec 18th 2015 Riddler: Which Geyser Gushes First? 
***

The Dec 18th 2015 Riddler from [FiveThirtyEight](https://fivethirtyeight.com/features/which-geyser-gushes-first/) asks the following: 

> You arrive at the beautiful Three Geysers National Park. You read a placard explaining that the three eponymous geysers — creatively named A, B and C — erupt at intervals of precisely two hours, four hours and six hours, respectively. However, you just got there, so you have no idea how the three eruptions are staggered. Assuming they each started erupting at some independently random point in history, _what are the probabilities_ that A, B and C, respectively, will be the first to erupt after your arrival?

**Simulation Approach**: We'll start by performing a Monte Carlo simulation to estimate the probability of each of the three geysers going off first after your arrival.  To do this we randomly sample your arrival time from $U[0,2]$ (we choose $2$ because it is guaranteed that at least one geyser will fire in any two hour window).  We then randomly sample firing times for Geysers $A,$ $B,$ and $C$ from $U[0,2]$, $U[0,4]$, and $U[0,6]$ respectively.  Finally, we record the number of times each geyser fires first and divide out by the total number of trials. 

In [1]:
using Distributions

In [2]:
function geyser_sim(M::Int64=1000000)
    # ------------------------------------------------------------
    # Function to simulate M trials of random trips to the Geyser  
    # ------------------------------------------------------------
    
    # Initialize counts of first geyser 
    counts = zeros(Int64, 3)
    
    for ii=1:(Int64(M/10000)+1)
        
        # Draw samples for your arrival from U[0,12]
        arrivals = rand(Uniform(0.0, 2.0), 10000)

        # Sample wait times from your arrival to first gush from each geyser 
        Awaits = map(x -> (x >= 0) ? x : x + 2.0, (rand(Uniform(0.0,2.0),10000)-arrivals))
        Bwaits = map(x -> (x >= 0) ? x : x + 4.0, (rand(Uniform(0.0,4.0),10000)-arrivals))
        Cwaits = map(x -> (x >= 0) ? x : x + 6.0, (rand(Uniform(0.0,6.0),10000)-arrivals))
        
        # Record which geyser goes off first for each trial 
        firsts  = map(abc -> findmin(abc)[2], zip(Awaits, Bwaits, Cwaits))

        # Count trials for which each geyser wins 
        for first in firsts
            counts[first] += 1
        end
    
    end
    
    # Divide counts by total trials to get probability estimates
    probs = counts / Float64(sum(counts)) 
    
    # Print probabilities 
    @printf "Pr(A) = %f, Pr(B) = %f, Pr(C) = %f\n" probs[1] probs[2] probs[3]
    
    return probs 
    
end

geyser_sim (generic function with 2 methods)

The estimated probabilities from the simulation with 100 Million trials are as follows: 

In [5]:
prob = geyser_sim(Int64(1e8));

Pr(A) = 0.638967, Pr(B) = 0.222179, Pr(C) = 0.138854


According to the estimates Geyser $A$ will be first around $64\%$ of the time, Geyser $B$ will be first $22\%$ of the time, and Geyser $C$ will be first $14\%$ of the time. 

**Analytic Approach**: OK, now let's try the pencil-and-paper approach and see if we get the same results. 

Notice that if we consider a $2$hr period directly after your arrival, Geyser $A$ will fire with probability 1, Geyser $B$ will fire with probability $\frac{1}{2}$, and Geyser $C$ will fire with probability $\frac{1}{3}$.  There are four possible cases for the two hour period: 

- **Case 1**: Geysers $A$, $B$, and $C$ all fire 
- **Case 2**: Geysers $A$ and $B$ fire, but not $C$ 
- **Case 3**: Geysers $A$ and $C$ fire, but not $B$ 
- **Case 4**: Geyser $A$ fires but both $B$ and $C$ do not 

Now, since the actual instances in time that a geyser fires is random, we assume that for each case, there is an equal probability that any of the actually-firing geysers will be the first to fire.  Under these assumptions we can calculate the probability that, say, Geyser $A$ will fire first by summing the product of the probability of each case occurring with the probability that Geyser $A$ fires first in each case. We have: 

\begin{eqnarray}
\nonumber
\textrm{Pr}(A) &=&
\left[\left(1\cdot\frac{1}{2}\cdot\frac{1}{3}\right)\cdot\frac{1}{3} \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{2}{3}\right)\cdot\frac{1}{2} \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{1}{3}\right)\cdot\frac{1}{2} \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{2}{3}\right)\cdot 1 \right]  = \frac{23}{36} = 0.63\bar{8} \\
&& \\
\textrm{Pr}(B) &=&
\left[\left(1\cdot\frac{1}{2}\cdot\frac{1}{3}\right)\cdot\frac{1}{3} \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{2}{3}\right)\cdot\frac{1}{2} \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{1}{3}\right)\cdot 0  \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{2}{3}\right)\cdot 0 \right]  = \frac{8}{36} = 0.\bar{2} \\
&& \\
\textrm{Pr}(C) &=&
\left[\left(1\cdot\frac{1}{2}\cdot\frac{1}{3}\right)\cdot\frac{1}{3} \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{2}{3}\right)\cdot 0 \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{1}{3}\right)\cdot \frac{1}{2}  \right] + 
\left[\left(1\cdot\frac{1}{2}\cdot\frac{2}{3}\right)\cdot 0 \right]  = \frac{5}{36} = 0.13\bar{8} \\
\end{eqnarray}

Notice that the simulation results obtained above are extremely close to the analytic results obtained here. 